# Ejercicio 1: 

Escribe un programa en Python que use Biopython para hacer una búsqueda BLAST de una secuencia de ADN que introduzcas por teclado. 
El programa debe mostrar por pantalla el número de resultados obtenidos, el E-value del mejor resultado y la descripción de la secuencia más similar. 
Hágalo de forma online y local.

### 1.1. Instalación de BLAST

1. Se descarga la versión para el equipo de BLAST desde el siguiente enlace: [BLAST](https://ftp.ncbi.nlm.nih.gov/blast/executables/blast+/LATEST/)
2. Se ejecuta, en nuestro caso, el *.exe* descargado y se instala en el equipo.
3. Se configura el *path* de BLAST como variable de entorno del sistema.
4. Se verifica la instalación con el comando `blastn --version`.

### 1.2. Secuencia de ADN por teclado

In [1]:
from debug_sequence.functions import check_sequence

In [2]:
def get_sequence():
    sequence = input("Introduce la secuencia de ADN: ").strip()
    sequence = check_sequence(sequence)
    return sequence

### 1.3. Implementación online

In [7]:
from Bio.Blast import NCBIWWW, NCBIXML

In [8]:
def search_online_sequence():
    sequence = get_sequence()

    result = NCBIWWW.qblast("blastn", "nt", sequence)
    blast_records = NCBIXML.read(result)
    
    if not blast_records.alignments:
        print("No se encontraron resultados.")
        
    num_results = len(blast_records.alignments)

    best_alignment = blast_records.alignments[0]
    best_hsp = best_alignment.hsps[0]
    e_value = best_hsp.expect
    description = best_alignment.title
    
    print("\nResultados BLAST:")
    print(f"Número de resultados: {num_results}")
    print(f"E-value del mejor resultado: {e_value}")
    print(f"Descripción de la secuencia más similar: {description}")

In [9]:
search_online_sequence()


Resultados BLAST:
Número de resultados: 50
E-value del mejor resultado: 2.08101e-37
Descripción de la secuencia más similar: gi|1584133852|gb|MK476408.1| Homo sapiens voucher Bantu_1_1 hemoglobin subunit beta (HBB) gene, complete cds


### 1.4. Implementación local

Para usar las bases de datos proporcionadas por el NCBI, es necesario descargarlas mediante el comando `perl update_blastdb.pl --decompress nt`, donde `nt` es la base de datos de nucleótidos. Para este ejercicio, se usará la base de datos `human_genome` y `nt`.

De la base de datos `nt` se han descargado los siguientes:

`perl "C:\Program Files\NCBI\blast-2.16.0+\bin\update_blastdb.pl" --decompress nt.000 nt.001 nt.002 nt.003 nt.004 nt.005 nt.006 nt.007 nt.008 nt.009 nt.010`

In [13]:
from Bio.Blast.Applications import NcbiblastnCommandline
from Bio.Blast import NCBIXML
import os
import warnings

warnings.simplefilter("ignore")
os.environ["BLASTDB"] = input("Introduce la ruta de la base de datos de BLAST: ")
os.environ["CMD"] = input("Introduce la ruta del ejecutable de BLAST: ")

r"C:\Users\Susana\Desktop\blast\human_genome\GCF_000001405.39_top_level"


r"C:\Usuarios\Susana\Escritorio\blast\nt"

r"C:\Archivos de programa\NCBI\blast-2.16.0+\bin\blastn.exe"

In [15]:
def search_local_sequence():
    sequence = get_sequence()

    fasta_file = "query.fasta"
    with open(fasta_file, "w") as f:
        f.write(">query\n")
        f.write(sequence)
    
    blastn_cline = NcbiblastnCommandline(
        cmd=os.environ["CMD"],
        query=fasta_file,
        db=os.environ["BLASTDB"],
        evalue=0.001,
        outfmt=5,
        out="results.xml"
    )

    blastn_cline()

    with open("results.xml") as result_handle:
        blast_records = NCBIXML.parse(result_handle)
        for blast_record in blast_records:
            if blast_record.alignments:
                print(f"Número de resultados: {len(blast_record.alignments)}")

                best_alignment = blast_record.alignments[0]
                best_hsp = best_alignment.hsps[0]
                print(f"E-value del mejor resultado: {best_hsp.expect}")
                print(f"Descripción del mejor resultado: {best_alignment.title}")
            else:
                print("No se encontraron coincidencias.")
    
    os.remove(fasta_file)
    os.remove("results.xml")

In [16]:
search_local_sequence()

ApplicationError: Non-zero return code 3 from '"C:\\Archivos de programa\\NCBI\\blast-2.16.0+\\bin\\blastn.exe" -out results.xml -outfmt 5 -query query.fasta -db C:\\Usuarios\\Susana\\Escritorio\\blast\\nt -evalue 0.001', message 'BLAST Database error: Database memory map file error'